# Đồ án cuối kỳ - Phân tích chủ đề văn bản
(Cập nhật 07/01/2021)

Nhóm: 12

Thành viên nhóm:
- Vũ Đăng Hoàng Long - MSSV: 18120203
- Nguyễn Huỳnh Đại Lợi - MSSV: 18120198

---
## Import thư viện

In [10]:
import pandas as pd
import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D

# Thư viện để lấy và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
!pip install pyvi ;
!pip install gensim ;
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
import gensim
import unicodedata

# Thư viện liên quan đến machine learning của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

ERROR: Invalid requirement: ''
You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
ERROR: Invalid requirement: ''
You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


---
## 1. Thu thập dữ liệu

Dữ liệu trong đồ án này được thu thập toàn bộ từ trang báo mạng Tuổi Trẻ Online (https://tuoitre.vn/).

Khi mới vào trang chủ, có thể thấy trang báo này phân bổ thứ tự và cấu trúc thông tin không đều, rất khó để có thể thu thập được. Tuy nhiên, may mắn là trang báo này có mục tin mới nhất chứa toàn bộ tin từ các chuyên mục và có tổ chức các mục có cấu trúc rõ ràng (Hình 1).

![Hình 1](resources/tin-moi-nhat.png "Hình 1")

Khi người dùng kéo xuống cuối trang, sẽ có nút "Xem thêm" (Hình 2). Người dùng nhấn vào đây và tin tức cũ hơn sẽ hiện ngay bên dưới để người dùng kéo xuống tiếp.

![Hình 2](resources/xem-them.png "Hình 2")

Thoáng nhìn qua, quy trình lướt tin tức sẽ bao gồm: Vào mục tin mới -> Kéo xuống cuối trang -> Bấm "Xem thêm" -> Kéo xuống cuối trang và lặp lại. Như vậy với cách làm đơn giản nhất là mô phỏng click chuột của người dùng để bấm nút xem thêm càng nhiều càng tốt, sau đó lấy file HTML của trang về và thực hiện parse (Đây chính là cách làm trong phiên bản đầu của quy trình này của nhóm). Tuy nhiên cách này rất thiếu hiệu quả bởi lẽ máy sẽ cần phải mở trình duyệt lên và mô phỏng thao tác của người dùng -> rất chậm do thời gian tải trang và tốn tài nguyên xử lý của máy tính do phải render trang web. Trong phiên bản đầu nhóm đã mất khoảng vài phút để có thể lấy được 480 tin.

Nhận thấy rằng trang tin này có một đặc điểm là có thể lăn chuột vô tận, nhóm tìm hiểu và phát hiện rằng có thể cải thiện hiệu suất của quy trình này lên rất nhiều lần! Cụ thể, khi người dùng bấm vào nút "Xem thêm", trang web sẽ gửi yêu cầu lên server và nhận về một file HTML có cấu trúc đơn giản chứa thông tin các bài báo (Hình 3). Sau đó trình duyệt sẽ thực hiện thay đổi DOM của trang web để thêm các tin này vào bên dưới, tạo hiệu ứng lăn chuột vô tận. Do vậy, thay vì phải đợi trình duyệt render trang web, nhóm có thể request thẳng đường link giống cách trang web request lên server. Việc làm này sẽ rút ngắn rất nhiều thời gian của quá trình thu thập dữ liệu!

![Hình 3](resources/xem-them-network.png "Hình 3")

Khi sử dụng công cụ theo dõi network của trình duyệt, nhóm phát hiện ra rằng đường link request tin mới của trang web khi bấm nút xem thêm lần đầu là "https://tuoitre.vn/timeline/0/trang-1.htm". Rất có thể khi thay thế trang-1 thành trang-2, trang-3,... chúng ta sẽ thu được các kết quả là tin mới của các lần bấm thứ 2, thứ 3,... vào nút "Xem thêm" (điều này đúng, nhóm đã kiểm chứng).

Đoạn code bên dưới sẽ thực hiện request vào đường link trên và thực hiện parse với BeautifulSoup như HTML bình thường để lấy đường link và chuyên mục của các tin tức. Sau đó sẽ thực hiện request trang tin chính từ đường link thu thập được và lấy các thông tin tiêu đề, mô tả và nội dung.

Kết quả sẽ được in bằng hàm tail() giúp có thể nắm được số lượng data lấy được thông qua index.

In [3]:
# Thiết lập đường dần cho phần 1
dir_1 = "src/scraped_data_demo"

In [56]:
# Thử nghiệm url đầu tiên
url = "https://tuoitre.vn/timeline/0/trang-1.htm"

# Lấy danh sách news_items
html_text = requests.get(url).text
html_tree = BeautifulSoup(html_text, 'html.parser')
news_items = html_tree.findAll('li', class_='news-item')

# Lấy ra link, title và category từ news_items
raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
for item in news_items:
    title = item.find('h3', class_='title-news').text.replace('\n','')
    link = "https://tuoitre.vn" + item.find('a').attrs["href"]
    category = item.find('a', class_='category-name').text
    raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)

raw_data.tail()

,links,title,description,content,class
15,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",NaN,NaN,Sức khỏe
16,https://tuoitre.vn/lang-kinh-24g-canh-giac-voi...,Lăng kính 24g: Cảnh giác với tội phạm trộm cắp...,NaN,NaN,Cần biết
17,https://tuoitre.vn/game-show-cuoi-cung-chi-tai...,Gameshow cuối cùng của Chí Tài - Cơ hội đổi đờ...,NaN,NaN,Giải trí
18,https://tuoitre.vn/giua-kho-khan-thoi-covid-19...,"Giữa khó khăn 'thời COVID-19', một doanh nghiệ...",NaN,NaN,Thời sự
19,https://tuoitre.vn/triet-pha-duong-day-mang-th...,"Triệt phá đường dây mang thai hộ, mỗi lần 'đẻ ...",NaN,NaN,Pháp luật


In [57]:
%%time
for _, row in raw_data.iterrows():
    news_page = requests.get(row["links"]).content
    news_tree = BeautifulSoup(news_page, "html.parser")
    # Lấy mô tả
    try:
        row["description"] = news_tree.find("h2", class_="sapo").text
    except:
        row["description"] = ''
    # Lấy nội dung
    try:
        body = news_tree.find("div", id="main-detail-body")
        content = body.findChildren("p", recursive=False)
        row["content"] = ""
        for x in content:
            row["content"] += x.text
    except:
        row["content"] = ''
    time.sleep(0.05)
    
raw_data.tail()

Wall time: 2.83 s


,links,title,description,content,class
15,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
16,https://tuoitre.vn/lang-kinh-24g-canh-giac-voi...,Lăng kính 24g: Cảnh giác với tội phạm trộm cắp...,,,Cần biết
17,https://tuoitre.vn/game-show-cuoi-cung-chi-tai...,Gameshow cuối cùng của Chí Tài - Cơ hội đổi đờ...,"TTO - Tối 5-1, tập đầu tiên của chương trình t...",Cơ hội đổi đời là chương trình truyền hình thự...,Giải trí
18,https://tuoitre.vn/giua-kho-khan-thoi-covid-19...,"Giữa khó khăn 'thời COVID-19', một doanh nghiệ...",TTO - Hầu hết doanh nghiệp tại Đồng Nai duy tr...,"Ngày 6-1, Sở Lao động - thương binh và xã hội ...",Thời sự
19,https://tuoitre.vn/triet-pha-duong-day-mang-th...,"Triệt phá đường dây mang thai hộ, mỗi lần 'đẻ ...",TTO - Công an quận Long Biên (Hà Nội) vừa triệ...,"Ngày 6-1, Công an quận Long Biên (Hà Nội) cho ...",Pháp luật


### Vấn đề phát sinh trong quá trình thu thập dữ liệu tự động hàng loạt

#### Thiếu dữ liệu làm đứt quãng quá trình thu thập
Do khối lượng dữ liệu có thể thu thập được rất lớn (ước tính lên đến hơn 500.000 tin), việc giám sát quá trình khai thác liên tục là không thể. Việc xuất hiện lỗi này đã khiến quá trình khai thác bị ngắt quãng mỗi khi xuất hiện. Nhóm đã khắc phục bằng cách đặt các khối lệnh ```try``` ```catch``` ở mỗi đoạn code parse các phần thông tin, tự động trả về rỗng nếu có lỗi.

#### Lỗi TooManyRedirects: Exceeded 30 redirects
Đây là lỗi xuất hiện khi một trang web tự điều hướng quá 30 lần (con số mặc định có thể chỉnh được).

Với một số server, khi được request sẽ tùy vào thông tin header của request (như trình duyệt, cache,...), server sẽ đính kèm lệnh chuyển hướng vào response và trình duyệt sẽ dùng thông tin đó điều hướng chuyển qua trang web nhất định. Chẳng hạn, khi người dùng điện thoại truy cập vào trang facebook.com, trình duyệt sẽ tự động điều hướng chuyển người dùng tới trang m.facebook.com là trang có giao diện dành cho điện thoại. Request HTTP trong python cũng không phải ngoại lệ khi có thiết lập mặc định là sẽ tự động chuyển hướng cho đến khi không còn nhận được lệnh từ server.

Khi tra cứu lỗi này trên google, nhóm nhận được các đề xuất giải quyết sau:
- Thêm tham số ```allow_redirects=True``` vào lệnh ```requests.get```
- Thêm tham số ```headers``` vào lệnh ```requests.get```. Tham số này chứa các thông tin mặc định của header mà trình duyệt gửi tới server.
- Tạo một ```session``` và thay đổi thuộc tính ```max_redirect``` lên hớn 30.

Với cách 1 vô nghĩa do tham số đó là mặc định sẵn, cách 3 thì không đảm bảo thay đổi bao nhiêu là đủ. Còn với cách 2, lệnh request cung cấp thêm thông tin cho server nhằm tránh tình trạng server không xác định được người dùng và rơi vào vòng lặp redirect vô tận. Tuy nhiên nhóm áp dụng và vẫn không khắc phục được. Khi thử lấy link này thì hoàn toàn không vô được.

Một điều thú vị là khi debug, nhóm phát hiện rằng các link sinh ra lỗi này đều có nội dung về giáo dục (Hình 4). Khi thử tìm kiếm trang riêng có đường link https://tuyensinh.tuoitre.vn/ thì được điều hướng trở về trang https://tuoitre.vn/giao-duc/tuyen-sinh.htm. Kết luận của nhóm: rất có thể trang báo này đã có quá trình thay đổi công nghệ ở một thời điểm nhất định trong quá khứ khiến cho các trang này không truy cập được -> không chỉ trang tin giáo dục mà còn có thể nhiều trang tin khác cũng bị tương tự.

![Hình 4](resources/tuyen-sinh-errors.png "Hình 4")

Quyết định của nhóm là sẽ áp dụng cách khắc phục 2 vì nó hợp lý và áp khối ```try``` ```catch``` điền thông tin trống với các link tiếp tục bị lỗi.

#### Lỗi 502 của trang web được request về
Lỗi này đến từ phía server của trang báo và xuất hiện rất ngẫu nhiên không xác định được. Nhóm đã giải quyết bằng cách đặt vòng lặp ```while``` tại các câu lệnh ```get``` và chỉ ngừng khi quá số lần cho phép hoặc nhận phản hồi 200 tức OK. Tuy nhiên cần chú ý cho phép cả phản hồi 301 vì đây là lệnh điều hướng trang có liên quan tới lỗi ở phần trước có đề cập.

### Thiết lập đoạn code thu thập dữ liệu tự động hàng loạt

In [59]:
def single_request_scraping(index = 1, sleep_time = 0.05):
    '''
    Thu thập các trang tin trong mục tin mới nhất của báo Tuổi Trẻ.
    Mỗi lần thu thập 20 tin. Sử dụng số index để thu thập các trang liên tiếp.
    Với index = 1 tức lấy 20 trang tin mới nhất.

    Lưu ý:
        Khi lấy liên tục cần chọn thời điểm
        vì có khả năng báo cập nhật tin tức mới sẽ làm 
        tin ở trang trước bị đẩy xuống trang sau.

    Param:
        index: số chỉ trang cần request.
    '''
    allow_status = [200, 301]
    s = requests.Session()
    s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    url = f"https://tuoitre.vn/timeline/0/trang-{index}.htm"

    # Lấy danh sách news_items
    try:
        response = s.get(url)
        try_left = limit_retry
        while (response.status_code not in allow_status and try_left > 0):
            print(f"Loi {response.status_code} tai trang {index}")
            response = s.get(url)
            try_left -= 1
        html_text = response.text
    except:
        print(f"Loi request tai trang {index}")
        return None
    
    html_tree = BeautifulSoup(html_text, 'html.parser')
    news_items = html_tree.findAll('li', class_='news-item')
    
    # Kiểm tra lỗi nếu không lấy được bất cứ item nào
    if (len(news_items) == 0):
        print(f'Trang {index} khong lay duoc item.')
        print(html_text)
        return None
    elif (len(news_items) != 20):
        print(f"Warning: Trang {index} chi lay duoc {len(news_items)} items.")
    
    # Lấy ra link, title và category từ news_items
    raw_data = pd.DataFrame(columns=["links","title","description","content","class"])
    for item in news_items:
        try:
            title = item.find('h3', class_='title-news').text.replace('\n','')
        except:
            title = ""
        link = "https://tuoitre.vn" + item.find('a').attrs["href"]
        try:
            category = item.find('a', class_='category-name').text
        except:
            category = ""
        raw_data = raw_data.append({"links":link, "title":title, "class":category}, ignore_index=True)
    
    # Tiến hành lấy nội dung từng link
    for _, row in raw_data.iterrows():
        try:
            response = s.get(row["links"])
            try_left = limit_retry
            while (response.status_code not in allow_status and try_left > 0):
                print(f"Loi {response.status_code} tai link {row['links']}")
                response = s.get(row["links"])
                try_left -= 1
            news_page = response.content
        except:
            print(f"Loi request tai link {row['links']}")
            row["description"] = ""
            row["content"] = ""
            continue
            
        news_tree = BeautifulSoup(news_page, "html.parser")
        # Lấy mô tả
        try:
            row["description"] = news_tree.find("h2", class_="sapo").text
        except:
            row["description"] = ''
        # Lấy nội dung
        try:
            body = news_tree.find("div", id="main-detail-body")
            content = body.findChildren("p", recursive=False)
            row["content"] = ""
            for x in content:
                row["content"] += x.text
        except:
            row["content"] = ''
        time.sleep(sleep_time)
    
    return raw_data

In [52]:
#Batch scraping
iter_num = 1  # Số batch bắt đầu
num = 200  # Số trang link trong 1 batch
continue_flag = True # Cờ hiệu kết thúc vòng lặp khi xảy ra lỗi

while (continue_flag):
    '''
    Khởi tạo dataframe rỗng. 
    Sau đó lấy đủ 1 số trang cho 1 batch.
    Rồi export file csv.
    '''
    batch_df = pd.DataFrame(columns=["links","title","description","content","class"])
    for index in range(iter_num*num+1,(iter_num+1)*num+1):
        data = single_request_scraping(index,0)
        if (data is None):
            continue_flag = False
            break
        print(f"Page {index} complete!")
        batch_df = batch_df.append(data)
    batch_df.to_csv(f'scraped_data\crawling_{iter_num}.csv',index=False,encoding="utf-8")    
    iter_num+=1

Page 3401 complete!
Page 3402 complete!
Page 3403 complete!
Page 3404 complete!
Page 3405 complete!
Page 3406 complete!
Page 3407 complete!
Page 3408 complete!
Page 3409 complete!
Page 3410 complete!
Page 3411 complete!
Page 3412 complete!
Page 3413 complete!
Page 3414 complete!
Page 3415 complete!
Page 3416 complete!
Page 3417 complete!
Page 3418 complete!
Page 3419 complete!
Page 3420 complete!
Page 3421 complete!
Page 3422 complete!
Page 3423 complete!
Page 3424 complete!
Page 3425 complete!
Page 3426 complete!
Page 3427 complete!
Page 3428 complete!
Page 3429 complete!
Page 3430 complete!
Page 3431 complete!
Page 3432 complete!
Page 3433 complete!
Page 3434 complete!
Page 3435 complete!
Page 3436 complete!
Page 3437 complete!
Page 3438 complete!
Page 3439 complete!
Page 3440 complete!
Page 3441 complete!
Page 3442 complete!
Page 3443 complete!
Page 3444 complete!
Page 3445 complete!
Page 3446 complete!
Page 3447 complete!
Page 3448 complete!
Page 3449 complete!
Page 3450 complete!


FileNotFoundError: [Errno 2] No such file or directory: 'scraped_data\\crawling_17.csv'

## 2. Khám phá dữ liệu

In [8]:
# Thiết lập đường dẫn thư mục để dùng trong phần 2
dir_2 = "src/raw_scraped_data/"

In [11]:
# Import thử 12 file đầu tiên
df = []
for i in tqdm(range(12)):
    df.append(pd.read_csv(dir_2 + f'crawling_{i}.csv'))
    
data_df=pd.concat(df)
data_df.reset_index(drop=True, inplace=True)
data_df.head()

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới


In [10]:
data_df.shape

(52000, 5)

In [11]:
# Các cột hiện có kiểu dữ liệu gì?
data_df.dtypes

links          object
title          object
description    object
content        object
class          object
dtype: object

In [12]:
data_df[data_df.duplicated(keep=False)].sort_values(by=['links'])

,links,title,description,content,class
48880,/dang-sai-lech-vu-dong-tam-facebooker-chuong-m...,"Đăng sai lệch vụ Đồng Tâm, Facebooker ‘Chương ...","TTO - Ngày 20-1, Cơ quan cảnh sát điều tra Côn...","Facebooker ""Chương May Mắn"" tên thật là Chung ...",Pháp luật
48879,/dang-sai-lech-vu-dong-tam-facebooker-chuong-m...,"Đăng sai lệch vụ Đồng Tâm, Facebooker ‘Chương ...","TTO - Ngày 20-1, Cơ quan cảnh sát điều tra Côn...","Facebooker ""Chương May Mắn"" tên thật là Chung ...",Pháp luật
307,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
254,https://tuoitre.vn/1-ca-covid-19-moi-phat-hien...,"1 ca COVID-19 mới, phát hiện sau 3 lần xét ngh...",TTO - Sau nhiều ngày số mắc mới khá cao và dồn...,"Theo Bộ Y tế, 1 ca mắc mới ngày 6-1 là ca nhập...",Sức khỏe
179,https://tuoitre.vn/affordable-luxury-dong-san-...,'Affordable luxury' - dòng sản phẩm đột phá về...,Apec Mandala Wyndham Mũi Né đánh dấu bước đầu ...,Apec Group và thương hiệu bất động sản mang tí...,Cần biết
...,...,...,...,...,...
229,https://tuoitre.vn/vuong-quoc-anh-ghi-nhan-hon...,Vương quốc Anh ghi nhận hơn 1.000 người chết m...,TTO - Vương quốc Anh ngày 6-1 ghi nhận thêm 1....,"Theo hãng tin Reuters, trong ngày 6-1, Vương q...",Sức khỏe
304,https://tuoitre.vn/wall-street-journal-bac-kin...,Wall Street Journal: Bắc Kinh ép Jack Ma chia ...,TTO - Chính quyền Trung Quốc đang cố gắng buộc...,Báo Wall Street Journal nhận định Jack Ma có r...,Kinh doanh
251,https://tuoitre.vn/wall-street-journal-bac-kin...,Wall Street Journal: Bắc Kinh ép Jack Ma chia ...,TTO - Chính quyền Trung Quốc đang cố gắng buộc...,Báo Wall Street Journal nhận định Jack Ma có r...,Kinh doanh
286,https://tuoitre.vn/xu-phat-3-nguoi-75-trieu-do...,Xử phạt 3 người 75 triệu đồng vì khai thác kho...,TTO - Chủ đất và hai người tham gia lấy phần c...,"Ngày 6-1, Công an tỉnh Trà Vinh tống đạt các q...",Pháp luật


In [14]:
data_df.isna().sum()

links            0
title          244
description    292
content        908
class            0
dtype: int64

In [22]:
data_df[data_df.isna().any(axis=1)].head()

,links,title,description,content,class
29,https://tuoitre.vn/lang-kinh-24g-xe-du-ben-coc...,"Lăng kính 24h: 'Xe dù, bến cóc' - vấn nạn nhiề...",NaN,NaN,Cần biết
123,https://tuoitre.vn/doc-bao-cung-ban-8-1-viet-n...,Đọc báo cùng bạn 8-1: Việt Nam sẽ có vắc xin d...,TTO - Chương trình hôm nay có nhiều thông tin ...,NaN,Bạn đọc làm báo
129,https://tuoitre.vn/video-daisuke-matsui-the-hi...,Video Daisuke Matsui thể hiện đẳng cấp ngay lầ...,NaN,NaN,Thể thao
137,https://tuoitre.vn/duoc-hlv-huynh-duc-kem-chat...,"Được HLV Huỳnh Đức 'kèm chặt', Đức Chinh liền ...","TTO - Chiều 7-1, tiền đạo Hà Đức Chinh đã gây ...",NaN,Thể thao
166,https://tuoitre.vn/lang-kinh-24g-doi-no-thue-h...,Lăng kính 24g: Đòi nợ thuê 'hết đất' lộng hành,NaN,NaN,Cần biết


In [29]:
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 2):
    display(data_df['class'].value_counts())

Thời sự                8529
Thế giới               6211
Kinh doanh             4589
Thể thao               4491
Pháp luật              4447
Sức khỏe               4271
Cần biết               2901
Giáo dục               2633
Văn hóa                1596
Nhịp sống trẻ          1580
Giải trí               1424
Khoa học               1397
Du Lịch                1334
Bạn đọc làm báo        1086
Công nghệ               855
Phóng sự                709
Doanh nghiệp            708
Nhà đất                 655
Giả - Thật              505
Tuyển sinh              388
Bình luận               372
Tài chính               159
Mua sắm                 145
Xe                      135
Xã hội                   93
Nhịp sống số             90
Đọc báo cùng bạn         76
Bút Bi                   58
Học đường                56
Tin tức                  55
Media                    49
Biết để khỏe             47
Đời sống                 43
Dự án                    38
Góc học tập              31
Sách                

In [24]:
data_df.drop(['links'],axis=1,inplace=True)
data_df.dropna(axis=0, how="any", inplace=True)
data_df['description']=data_df['description'].str.replace("TTO - ",'')
data_df['description']=data_df['description'].str.replace("TTO – ",'')

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...,...
51995,/tham-sat-o-thai-nguyen-nghi-pham-thua-nhan-nh...,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
51996,/mobifone-dat-loi-nhuan-hon-6-000-ty-dong-tron...,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
51997,/chuan-bi-co-le-hoi-hoa-anh-dao-tai-tp-hcm-201...,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
51998,/xet-xu-vu-nu-sinh-giao-ga-bui-van-cong-lien-t...,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


In [ ]:
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    if (type(txt) is not str):
        return txt
    return unicodedata.normalize('NFC', txt)

In [ ]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    
    # Tách dấu ra khỏi từ và lưu lại vị trí nguyên âm
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1: # Phụ âm thì bỏ qua
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
                
        if y != 0: # Nếu có dấu
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
            
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            # Trường hợp chữ "gì"
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    # Từ hợp lệ là từ không có phụ âm bị kẹp giữa nguyên âm
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, _ = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if (index - nguyen_am_index) != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
    :param sentence:
    :return:
    """
    if (type(sentence) is not str):
        return sentence
    
    words = sentence.split()
    for index, word in enumerate(words):
        # Tách chữ có dính dấu ngăn cách câu để chuẩn hóa
        cw = re.sub(r'(^\p{P}*)([\p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        # Chuẩn hóa nếu chữ không có vấn đề (thường fail khi chưa tách thành công các từ)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

In [ ]:
total_files = 218
text_attrs = ["title","description","content","class"]

for index in tqdm(range(total_files)):
    df=pd.read_csv(f'scraped_data/crawling_{index}.csv')
    # Xóa dòng thiếu
    df=df[~((df['title'].isnull())|
                  (df['description'].isnull())|
                  (df['content'].isnull()))]
    for attr in text_attrs:
        df[attr]=df[attr].apply(covert_unicode)
        df[attr]=df[attr].str.lower()
        df[attr]=df[attr].apply(chuan_hoa_dau_cau_tieng_viet)
    df.to_csv(f'scraped_data/crawling_{index}.csv', index=False)

In [ ]:
def lower_character(a):
    return a.lower()
data_df['content']=data_df['content'].apply(lower_character)
#xoá ký tự thừa
def remove_unnecessary(paragraph):
    temp=re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',paragraph)
    return re.sub(r'\s+', ' ', temp).strip()

data_df['content']=data_df['content'].apply(remove_unnecessary)

In [ ]:
with open('vietnamese-stopwords-dash.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    
lines

In [ ]:
# Danh sách stopword
stopword = set(lines)
 
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

data_df['content']=data_df['content'].apply(remove_stopwords)

In [ ]:
data_df['content']=data_df['content'].apply(ViTokenizer.tokenize)

In [ ]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)

In [ ]:
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopwords:
            words.append(word)
    return ' '.join(words)


In [ ]:
train_X_df=train_X_df.apply(remove_stopwords)
val_X_df=val_X_df.apply(remove_stopwords)

# Feature Engineering đưa dữ liệu dạng văn bản đã được xử lý về dạng vector thuộc tính có dạng số học bằng TF-IDF Vectors ở mức Word level 

In [ ]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(train_X_df) # learn vocabulary and idf from training set
train_X_tfidf =  tfidf_vect.transform(train_X_df)
# assume that we don't have test set before
val_X_tfidf =  tfidf_vect.transform(val_X_df)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(train_X_tfidf)


train_X_tfidf_svd = svd.transform(train_X_tfidf)
val_X_tfidf_svd = svd.transform(val_X_tfidf)

In [66]:
train_X_tfidf_svd

array([[ 0.20030094,  0.18793731, -0.12534865, ..., -0.00110729,
        -0.0142511 , -0.01850076],
       [ 0.11525002, -0.0581487 , -0.02183506, ...,  0.03544429,
        -0.01520727,  0.01627132],
       [ 0.18858558, -0.08719651, -0.08031146, ..., -0.01831743,
        -0.00367883,  0.00197929],
       ...,
       [ 0.12423795,  0.09512202, -0.07983218, ...,  0.02051833,
         0.00683792,  0.02198647],
       [ 0.12335633, -0.00665921, -0.05612389, ...,  0.06285094,
        -0.01132967,  0.0136735 ],
       [ 0.19979618, -0.01186973,  0.0662302 , ...,  0.01725242,
        -0.02033546,  0.00591102]])

# Xây dựng mô hình

In [37]:
encoder =LabelEncoder()
train_y_sr_n = encoder.fit_transform(train_y_sr)
val_y_sr_n = encoder.fit_transform(val_y_sr)

array(['cần biết', 'giáo dục', 'kinh doanh', 'nhịp sống trẻ', 'pháp luật',
       'sức khỏe', 'thế giới', 'thể thao', 'thời sự', 'văn hóa'],
      dtype=object)

# Logistic Regression

In [72]:
from sklearn.linear_model import LogisticRegression
log_reg_model=LogisticRegression()
log_reg_model.fit(train_X_tfidf_svd,train_y_sr_n)


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### Độ chính xác tập training

In [73]:
log_reg_model.score(train_X_tfidf_svd,train_y_sr_n)

0.8335266303862967

### Độ chính xác tập valuation

In [74]:
prediction=log_reg_model.predict(val_X_tfidf_svd)

In [77]:
1 - (prediction != val_y_sr_n).mean()

0.8116545265348596

# Support Vector Machine (SVM)

In [80]:
from sklearn.svm import SVC
svm_classifier=SVC()
svm_classifier.fit(train_X_tfidf_svd,train_y_sr_n)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

### Độ chính xác tập training

In [81]:
svm_classifier.score(train_X_tfidf_svd,train_y_sr_n)

0.27085377821393525

### Độ chính xác tập valuation

In [82]:
prediction=svm_classifier.predict(val_X_tfidf_svd)
1 - (prediction != val_y_sr_n).mean()

0.2663891779396462

# MLPClassifier

In [84]:
mlp_classifier=MLPClassifier(hidden_layer_sizes=(20), activation='tanh',
                                 solver='lbfgs', random_state=0, max_iter=2500)
mlp_classifier.fit(train_X_tfidf_svd,train_y_sr_n)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=20, learning_rate='constant',
              learning_rate_init=0.001, max_iter=2500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=0, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

### Độ chính xác tập training

In [85]:
mlp_classifier.score(train_X_tfidf_svd,train_y_sr_n)

1.0

### Độ chính xác tập valuation

In [86]:
prediction=mlp_classifier.predict(val_X_tfidf_svd)
1 - (prediction != val_y_sr_n).mean()

0.7835587929240375